<a href="https://colab.research.google.com/github/madhumitafilos/Collab/blob/main/Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 import the TensorFlow library and fixes the random seed.

In [89]:
# numerical library:
import numpy as np

# data manipulation library:
import pandas as pd

# to split data to training and validation dataset
from sklearn.model_selection import train_test_split

# import tensorflow
import tensorflow as tf
from tensorflow import keras

#Install and import keras tuner
%pip install -q -U keras-tuner
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from keras_tuner import RandomSearch, Hyperband, BayesianOptimization
from keras_tuner.engine.hypermodel import HyperModel
from keras_tuner.engine.hyperparameters import HyperParameters

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

Read source data

In [13]:
wind_DS = pd.read_csv('/content/drive/MyDrive/training_data.csv')
wind_DS.head()

,active_power,timestamp,pitch_angle,reactive_power,nacelle_angle,nacelle_temp,wind_speed1,wind_speed2,wind_speed_avg,wind_angle,...,outdoor_temp,rotor_angular_velocity,rotor_bearing_temp,weather_temp,pressure,humidity,weather_wind_speed,weather_wind_angle,rain_1h,snow_1h
0,801.22998,2013-01-01 00:00:00,-1.0,67.559998,286.00000,20.129999,7.52,7.76,7.64,286.19000,...,5.44,16.950001,26.049999,5.39,1011.0,75.0,5.66,180.0,0.0,0.0
1,943.16998,2013-01-01 00:10:00,-1.0,70.260002,286.00000,21.420000,8.18,8.45,8.31,288.32999,...,5.74,17.139999,26.100000,5.39,1011.0,75.0,5.66,180.0,0.0,0.0
2,998.48999,2013-01-01 00:20:00,-1.0,75.330002,286.00000,22.049999,8.29,8.66,8.47,293.04001,...,6.09,17.150000,26.219999,5.39,1011.0,75.0,5.66,180.0,0.0,0.0
3,837.96002,2013-01-01 00:30:00,-1.0,82.739998,286.00000,22.299999,7.89,8.24,8.06,294.01999,...,6.35,16.910000,26.309999,5.39,1011.0,75.0,5.66,180.0,0.0,0.0
4,871.57001,2013-01-01 00:40:00,-1.0,82.349998,294.17999,22.600000,7.86,8.20,8.03,299.22000,...,6.51,16.920000,26.389999,5.39,1011.0,75.0,5.66,180.0,0.0,0.0


In [14]:
#X = wind_DS[['pitch_angle','reactive_power','nacelle_angle','nacelle_temp','wind_speed1','wind_speed2','wind_speed_avg','wind_angle','vane_angle','outdoor_temp','rotor_angular_velocity','rotor_bearing_temp','weather_temp','pressure','humidity','weather_wind_speed','weather_wind_angle','rain_1h','snow_1h']]
X = wind_DS[['pitch_angle','reactive_power','nacelle_temp','wind_speed_avg','rotor_angular_velocity']]
y = wind_DS['active_power']

In [115]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 100)

In [108]:
# Scale network input data.
scaler = preprocessing.StandardScaler()
Xn = scaler.fit_transform(X)

# Scale test set.
Xtn = scaler.fit_transform(test_X)  #apply scaling to test data

In [109]:
train_X.shape , test_X.shape , train_y.shape , test_y.shape

((146237, 5), (62673, 5), (146237,), (62673,))

Build model that consists of three hidden layers and an output layer with a softmax activation function. We will use the Adam optimizer and categorical cross-entropy loss function.

In [110]:
from tensorflow.keras.optimizers import Adam

opt_1 = Adam(learning_rate=0.001)

In [126]:
z = tf.reshape(X, (1044550  ,1))
model_1 = Sequential([layers.Input((5,1)),
                     #layers.Lambda(lambda x: x / 255),
                     layers.Flatten(),
                     layers.Dense(10, activation='softmax')])

model_1.compile(optimizer=opt_1, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_1.fit(x=train_X, y=train_y, validation_data=(test_X, test_y), epochs=10)

Epoch 1/10


KeyError: ignored

In [86]:
def build_model(hp):
    model = keras.Sequential()

    model.add(layers.Flatten(input_shape=(None, 5)))
    #z = tf.reshape(X, (208910, 5))
    model.add(layers.Flatten())

    # Tune the number of units in the first dense layer
    # Choose an optimal value between 32-512
    hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units1, activation='relu'))

    # Tune the number of units in the second dense layer
    # Choose an optimal value between 32-512
    hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units2, activation='relu'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])

    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [79]:
  print(tf.shape(X))

tf.Tensor([208910      5], shape=(2,), dtype=int32)


Create the tuner

In [87]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='Windproject')

Reloading Tuner from my_dir/Windproject/tuner0.json


In [88]:
tuner.search(train_X, train_y, epochs=5, validation_data=(test_X, test_y))


Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
256               |256               |units1
160               |320               |units2
0.01              |0.001             |learning_rate



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 238, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 232, in _build_and_fit_model
    model = self._try_build(hp)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 164, in _try_build
    model = self._build_hypermodel(hp)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 155, in _build_hypermodel
    model

RuntimeError: ignored

In [20]:
# initialize our model with the default hyperparameter values
print("[INFO] initializing model...")
# model = get_mlp_model()
# train the network (i.e., no hyperparameter tuning)
print("[INFO] training model...")
H = mlp.fit(Xn, train_y,
	validation_data=(Xtn, test_y),
	batch_size=8,
	epochs=20)
# make predictions on the test set and evaluate it
print("[INFO] evaluating network...")
accuracy = model.evaluate(Xtn, test_y)[1]
print("accuracy: {:.2f}%".format(accuracy * 100))

[INFO] initializing model...
[INFO] training model...


TypeError: ignored